# Analyzing Used Car Data on Ebay Kleinanzeigen

We will be working with a dataset of used cars from *ebay Kleinanzeigen*, a german classifieds section of the eBay website.

Our dataset was prepared by [Dataquest](www.dataquest.io), which took a sample of 50,000 data points from the original [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data) dataset, and simulated a less-clean version of dataset.

The Data dictionary provided with the data consists of: 

    dateCrawled - When this ad was first crawled. All field-values are taken from this date.
    name - Name of the car.
    seller - Whether the seller is private or a dealer.
    offerType - The type of listing
    price - The price on the ad to sell the car.
    abtest - Whether the listing is included in an A/B test.
    vehicleType - The vehicle Type.
    yearOfRegistration - The year in which the car was first registered.
    gearbox - The transmission type.
    powerPS - The power of the car in PS.
    model - The car model name.
    kilometer - How many kilometers the car has driven.
    monthOfRegistration - The month in which the car was first registered.
    fuelType - What type of fuel the car uses.
    brand - The brand of the car.
    notRepairedDamage - If the car has a damage which is not yet repaired.
    dateCreated - The date on which the eBay listing was created.
    nrOfPictures - The number of pictures in the ad.
    postalCode - The postal code for the location of the vehicle.
    lastSeenOnline - When the crawler saw this ad last online.
    
The goal of this project is to clean the data and analyze the included used car listings.


In [19]:
import numpy as np
import pandas as pd

autos = pd.read_csv("autos.csv", encoding='Latin-1')

In [20]:
#Investigation
autos
autos.info()
autos.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Our dataset contains 20 columns, most of which are stored as strings. There are 5 columns stored numerically as int, and some columns with datetime data stored as strings. In addition, there are a few columns with incomplete fields: vehicleType, gearbox,model,fuelType,notRepairedDamage

We'll start by cleaning the column names to make the data easier to work with.

# Clean Columns

In [21]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Changes we'll make to the column labels:
- camelcase to snakecase  
- rename some columns for readability 

In [22]:
new_cols = []
spec_dict = {'yearOfRegistration':'registration_year', 'monthOfRegistration':'registration_month',
                  'notRepairedDamage':'unrepaired_damage', 'dateCreated':'ad_created'}
auto_col= autos.columns

for c in auto_col:
    new_str = ""
    if c in spec_dict:
        new_cols.append(spec_dict[c])
    else:
        for s in c:
            if s.isupper():
                new_str += '_'
                new_str += s.lower()
            else:
                new_str += s
        new_cols.append(new_str)

autos.columns = new_cols
autos.head(3)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


In [23]:
#Investigation
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 19:48:02,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Cols to drop:
    offer_type (4999 instances of Angebot),
    seller (4999 instances of privat)

Cols to investigate(missing fields):
    - vehicle_type,
    gearbox,
    vehicle_type,
    model,
    fuel_type,
    unrepaired_damage

Cols to clean:
    price,
    odometer

In [24]:
#Data Cleaning
autos["model"].value_counts()
autos["fuel_type"].value_counts()
autos["unrepaired_damage"].value_counts()
red_size = autos[(autos["model"].notna()) 
                & (autos["gearbox"].notna()) 
                & (autos["vehicle_type"].notna()) 
                & (autos["fuel_type"].notna())  
#                 & (autos["unrepaired_damage"].notna())
                ].shape[0]
print('reduced size = ', red_size)
red_size / 50000


reduced size =  40346


0.80692

candidates for boolean mask field removal:
- vehicle_type, gearbox, model, fueltype
- removing these would retain 80% of dataset

In [25]:
#Clean Numeric Columns
autos["price"] = autos["price"].str.lstrip('$').str.replace(",","").astype(int)
autos["odometer"] = autos["odometer"].str.rstrip('km').str.replace(",","").astype(int)
autos.rename(columns={'odometer':'odometer_km'}, inplace=True)

In [26]:
# Investigation - price
autos["price"].describe()
autos["price"].min()
autos["price"].value_counts().sort_index(ascending=True)

overpriced_autos = autos[autos["price"].between(100000,10000000)]

print('overpriced (>100,000) autos are', str(overpriced_autos.shape[0]/autos.shape[0]*100), '% of listings')
# investigation - odometer_km
# autos["odometer_km"].unique()

overpriced (>100,000) autos are 0.092 % of listings


In [27]:
# cleaning
autos = autos[autos["price"].between(100,100000)]
print(autos["price"].describe())

count    48185.000000
mean      5796.099741
std       7525.532405
min        100.000000
25%       1250.000000
50%       3000.000000
75%       7499.000000
max      99900.000000
Name: price, dtype: float64


removed price entries which were outside of realistic range (less than 100, more than 400,000)
no odometer entries were suspicious

Since there are so few unique values in 'odometer_km' col, we can safely assume that the ebay listing uses a category scheme to estimate the selling vehicles odometer values.
For price column, while there are some fields which are extremely high, limiting the price below that does not significantly change the averages: most cars are sold somewhere between 1,000 and 10,000 dollars

In [28]:
#date_crawled
autos['date_crawled'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.025340
2016-03-06    0.014050
2016-03-07    0.036090
2016-03-08    0.033164
2016-03-09    0.033019
2016-03-10    0.032313
2016-03-11    0.032624
2016-03-12    0.036920
2016-03-13    0.015690
2016-03-14    0.036692
2016-03-15    0.034305
2016-03-16    0.029470
2016-03-17    0.031504
2016-03-18    0.012867
2016-03-19    0.034762
2016-03-20    0.037813
2016-03-21    0.037190
2016-03-22    0.032811
2016-03-23    0.032292
2016-03-24    0.029449
2016-03-25    0.031504
2016-03-26    0.032292
2016-03-27    0.031109
2016-03-28    0.034949
2016-03-29    0.034139
2016-03-30    0.033703
2016-03-31    0.031856
2016-04-01    0.033662
2016-04-02    0.035633
2016-04-03    0.038601
2016-04-04    0.036567
2016-04-05    0.013054
2016-04-06    0.003175
2016-04-07    0.001390
Name: date_crawled, dtype: float64

Dates crawled between 03/05 and 04/07

In [29]:
autos['last_seen'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.001079
2016-03-06    0.004317
2016-03-07    0.005437
2016-03-08    0.007326
2016-03-09    0.009567
2016-03-10    0.010646
2016-03-11    0.012411
2016-03-12    0.023804
2016-03-13    0.008882
2016-03-14    0.012639
2016-03-15    0.015876
2016-03-16    0.016437
2016-03-17    0.028121
2016-03-18    0.007305
2016-03-19    0.015773
2016-03-20    0.020650
2016-03-21    0.020546
2016-03-22    0.021376
2016-03-23    0.018574
2016-03-24    0.019736
2016-03-25    0.019114
2016-03-26    0.016623
2016-03-27    0.015544
2016-03-28    0.020836
2016-03-29    0.022310
2016-03-30    0.024717
2016-03-31    0.023846
2016-04-01    0.022870
2016-04-02    0.024883
2016-04-03    0.025132
2016-04-04    0.024551
2016-04-05    0.124935
2016-04-06    0.221999
2016-04-07    0.132137
Name: last_seen, dtype: float64

In [30]:
autos['ad_created'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000042
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000042
2016-02-05    0.000042
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000042
2016-02-14    0.000042
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000042
2016-02-19    0.000062
2016-02-20    0.000042
2016-02-21    0.000062
                ...   
2016-03-09    0.033122
2016-03-10    0.032022
2016-03-11    0.032936
2016-03-12    0.036754
2016-03-13    0.017059
2016-03-14    0.035322
2016-03-15    0.034035
2016-03-16    0.029968
2016-03-17    0.031151
2016-03-18    0.013573
2016-03-19    0.033641
2016-03-20    0.037875
2016-03-21 

Ads created between 06/11 2015, 04/07 2019

In [31]:
autos["registration_year"].describe()
autos["registration_year"].value_counts().sort_index()

1000       1
1001       1
1111       1
1800       2
1910       2
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       1
1951       1
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       4
1959       6
1960      22
1961       6
1962       4
1963       8
1964      12
        ... 
2000    3103
2001    2628
2002    2477
2003    2693
2004    2699
2005    2911
2006    2668
2007    2272
2008    2208
2009    2080
2010    1583
2011    1617
2012    1305
2013     797
2014     660
2015     376
2016    1196
2017    1383
2018     468
2019       2
2800       1
4100       1
4500       1
4800       1
5000       3
5911       1
6200       1
8888       1
9000       1
9999       3
Name: registration_year, Length: 95, dtype: int64

There is some bad data for some of these entries.
To clean, setting the start year as 1900, end year as 2016 as:

    a) commercial cars were invented around 1900
    b) the data was collected in 2016

In [32]:
print('percent dataset removed outside 1900-2016 =', ((autos['registration_year'].shape[0] - autos[autos['registration_year'].between(1900,2017)]['registration_year'].shape[0]) / autos['registration_year'].shape[0] * 100), '%')


percent dataset removed outside 1900-2016 = 1.0148386427311404 %


In [33]:
autos['registration_year'] = autos[autos['registration_year'].between(1900,2017)]['registration_year']
autos['registration_year'].value_counts(normalize=True).head(10)


2000.0    0.065058
2005.0    0.061032
1999.0    0.060361
2004.0    0.056588
2003.0    0.056462
2006.0    0.055938
2001.0    0.055099
2002.0    0.051933
1998.0    0.048998
2007.0    0.047635
Name: registration_year, dtype: float64

Most cars listed are around 10-15 years old, as most registration years are from early 2000's

### Brand Exploration

In [34]:
brand_list = []
print("total =",autos["brand"].shape[0])
print("estimated german shareholdings = ", "{0:.2f}".format(100 * autos["brand"].value_counts().head(5).sum() / (autos["brand"].shape[0])), "%")
print(autos["brand"].value_counts().head(20))
for i, count in enumerate(autos["brand"].value_counts()):
    if count / autos["brand"].shape[0] > .01:
        brand_list.append(autos["brand"].value_counts().index[i])
print(brand_list)


total = 48185
estimated german shareholdings =  61.23 %
volkswagen        10271
bmw                5240
opel               5223
mercedes_benz      4625
audi               4146
ford               3355
renault            2306
peugeot            1420
fiat               1252
seat                913
skoda               775
nissan              739
mazda               736
smart               690
citroen             682
toyota              611
hyundai             479
sonstige_autos      450
volvo               435
mini                417
Name: brand, dtype: int64
['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault', 'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart', 'citroen', 'toyota']


Of the brands being sold, volkswagen is by far the most popular, making up almost a fifth of the market, and the 5 german manufacturers topping the list making up *61% of total listings!*
For our aggregate analysis, we are choosing cars with a > 1% marekt share in the listings

In [35]:
brand_mean_price = {}
for brand in brand_list:
    brand_mean_price[brand] = autos[autos['brand'] == brand]["price"].mean() 

upper = 5000
lower = 3000
sorted(((value,key) for (key,value) in brand_mean_price.items() \
#         if value > lower and value < upper \
       ), reverse=True)

[(9219.534249879402, 'audi'),
 (8484.146378378378, 'mercedes_benz'),
 (8181.499236641222, 'bmw'),
 (6394.309677419355, 'skoda'),
 (5366.032713465096, 'volkswagen'),
 (5148.0032733224225, 'toyota'),
 (4681.94046008119, 'nissan'),
 (4348.652792990142, 'seat'),
 (4075.319293478261, 'mazda'),
 (3783.6788856304984, 'citroen'),
 (3719.3636363636365, 'ford'),
 (3538.344927536232, 'smart'),
 (3086.930281690141, 'peugeot'),
 (2974.764503159104, 'opel'),
 (2815.635782747604, 'fiat'),
 (2450.9015611448394, 'renault')]

Of the cars being analyzed, audis, mercedes and bmws are sold at the highest mean prices, reflecting their status as luxury german car brands. Of the more affordable car brands, volkswagen, toyatas, and nissans hold their value the best. Brands like Fort, smart and citroen price the lowest out of major car brands.

In [36]:
brand_mean_price = {}
brand_mean_mileage = {}
for brand in brand_list:
    brand_mean_mileage[brand] = autos[autos['brand'] == brand]["odometer_km"].mean() 
    brand_mean_price[brand] = autos[autos['brand'] == brand]["price"].mean() 

bmp_series = pd.Series(brand_mean_price)
bmm_series = pd.Series(brand_mean_mileage)

df = pd.DataFrame(bmp_series, columns=['mean_price'])
df['mean_mileage_km'] = bmm_series

import pprint
pp = pprint.PrettyPrinter()
pp.pprint(df.sort_values(by=['mean_mileage_km'], ascending=False))

                mean_price  mean_mileage_km
bmw            8181.499237    132884.541985
mercedes_benz  8484.146378    131088.648649
audi           9219.534250    129634.587554
opel           2974.764503    129442.848937
volkswagen     5366.032713    129008.859897
renault        2450.901561    128261.058109
peugeot        3086.930282    127316.901408
mazda          4075.319293    124959.239130
ford           3719.363636    124383.010432
seat           4348.652793    122020.810515
citroen        3783.678886    119816.715543
nissan         4681.940460    118707.713126
fiat           2815.635783    117408.146965
toyota         5148.003273    116219.312602
skoda          6394.309677    111051.612903
smart          3538.344928    100833.333333


According to the above table, BMW, mercedes and audi brands have the highest average mileage, and still command the highest resale value, again reflecting their status as luxury brands. Opel is the cheapest brand with the highest mileage, and Skoda resales for the most with the fewest amount of miles. 